# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **weather phenomena** (see below) for the region of **Saint Petersburg, St.-Petersburg, Russia**, or **Russia** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Saint Petersburg, St.-Petersburg, Russia** to Ann Arbor, USA. In that case at least one source file must be about **Saint Petersburg, St.-Petersburg, Russia**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Saint Petersburg, St.-Petersburg, Russia** and **weather phenomena**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **weather phenomena**?  For this category you might want to consider seasonal changes, natural disasters, or historical trends.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [1]:
!conda install -y beautifulsoup4 lxml requests folium

Fetching package metadata .........
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /opt/conda:
#
beautifulsoup4            4.6.0                    py35_0    conda-forge
folium                    0.3.0                    py35_0    conda-forge
lxml                      3.8.0                    py35_0    conda-forge
requests                  2.18.4                   py35_1    conda-forge


In [2]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colors import rgb2hex
from matplotlib import cm
import branca.colormap as cmap

import matplotlib as mpl
import mplleaflet
import folium
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

%matplotlib inline

In [3]:
def scrape_table(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data)
    
    rows = soup.find_all('table')[1].find_all('tr')[4:]
    nrows = len(rows)
    ncols = len(rows[0].find_all('td'))
    colnames = [
        'Year', 'Month', 'Day', 'Hr', 'Mn', 'Sec', 'Tsu', 'Vol', 'Addl EQ Info', 
        'Name', 'Lat', 'Lon', 'Focal Depth', 'Mag', 'MMI int', 
        'Deaths', 'Deaths Num Cat', 'Injuries', 'Injuries Num Cat', 
        'Dmg $Mill', 'Dmg $Mill Cat', 'Houses Destroyed', 'Houses Destroyed Cat', 
        'Houses Damaged', 'Houses Damaged Cat', 'Photos'
    ]
    df = np.zeros((nrows, ncols))
    df = pd.DataFrame(df, columns=colnames)

    for row in range(nrows):

        for col in range(ncols):
            value_lst = rows[row].find_all('td')[col].contents

            try:
                df.iloc[row, col] = value_lst[0]
                
            except:
                continue
                
    return df

In [4]:
def make_data_frame(url_list):
    
    for i, url in enumerate(url_list):
        
        if i == 0:
            DF = scrape_table(url)
            DF['Country'] = i
            
        else:
            df = scrape_table(url)
            df['Country'] = i
            DF = pd.concat([DF, df], axis=0)
            
    return DF

In [5]:
def preprocess_raw_data(raw_data):
    df = raw_data[['Year', 'Month', 'Name', 'Lat', 'Lon', 'Mag', 'Country']]
    df[['Year', 'Month']] = df[['Year', 'Month']].astype('int')
    df[['Lat', 'Lon']] = df[['Lat', 'Lon']].astype('float')
    df['Mag'] = df['Mag'].astype('float')
    
#     np.random.uniform(-1, 1)
    mask = (df['Lat'] < 51) & (df['Lat'] > -10)

    return df[mask]

In [6]:
url_1 = 'https://www.ngdc.noaa.gov/nndc/struts/results?bt_0=1900&st_0=&type_17=EXACT&query_17=40&op_12=eq&v_12=&type_12=Or&query_14=None+Selected&type_3=Like&query_3=&st_1=&bt_2=&st_2=&bt_1=&bt_4=0.0&st_4=9.9&bt_5=&st_5=&bt_6=&st_6=&bt_7=&st_7=&bt_8=&st_8=&bt_9=&st_9=&bt_10=&st_10=&type_11=Exact&query_11=&type_16=Exact&query_16=&bt_18=&st_18=&ge_19=&le_19=&type_20=Like&query_20=&display_look=1&t=101650&s=1&submit_all=Search+Database'
url_2 = 'https://www.ngdc.noaa.gov/nndc/struts/results?bt_0=1900&st_0=&type_17=EXACT&query_17=30&op_12=eq&v_12=&type_12=Or&query_14=None+Selected&type_3=Like&query_3=&st_1=&bt_2=&st_2=&bt_1=&bt_4=0.0&st_4=9.9&bt_5=&st_5=&bt_6=&st_6=&bt_7=&st_7=&bt_8=&st_8=&bt_9=&st_9=&bt_10=&st_10=&type_11=Exact&query_11=&type_16=Exact&query_16=&bt_18=&st_18=&ge_19=&le_19=&type_20=Like&query_20=&display_look=1&t=101650&s=1&submit_all=Search+Database'
url_3 = 'https://www.ngdc.noaa.gov/nndc/struts/results?bt_0=1900&st_0=&type_17=EXACT&query_17=60&op_12=eq&v_12=&type_12=Or&query_14=None+Selected&type_3=Like&query_3=&st_1=&bt_2=&st_2=&bt_1=&bt_4=0.0&st_4=9.9&bt_5=&st_5=&bt_6=&st_6=&bt_7=&st_7=&bt_8=&st_8=&bt_9=&st_9=&bt_10=&st_10=&type_11=Exact&query_11=&type_16=Exact&query_16=&bt_18=&st_18=&ge_19=&le_19=&type_20=Like&query_20=&display_look=1&t=101650&s=1&submit_all=Search+Database'
url_list = [url_1, url_2, url_3]
# url_list = [url_1]

raw_data = make_data_frame(url_list)

data = preprocess_raw_data(raw_data)

/opt/conda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /opt/conda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))
/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/opt/conda/lib/

In [15]:
def draw_map(data):
    MAP = folium.Map(location=[25, 100], min_zoom=3, zoom_start=3, 
                     max_zoom=10, max_bounds=False)
    
    colormap = cmap.linear.OrRd.scale(data['Mag'].min(), data['Mag'].max())
    colormap.caption = 'Magnitude'
    MAP.add_child(colormap)
    
    data['For Popup'] = data.apply(lambda row: '{} ({}/{}); Mag: {}'.format(row['Name'], 
                                                                            row['Month'], 
                                                                            row['Year'],
                                                                            row['Mag']), axis=1)
    data['Intensity'] = (data['Mag']-data['Mag'].min()) / (data['Mag'].max()-data['Mag'].min())
    data['Radius'] = 0.1*(1.9**data['Mag'])
    
    data.apply(lambda row: folium.CircleMarker(location=[row['Lat'], row['Lon']], 
                                               radius=row['Radius'],
                                               fill_color=rgb2hex(cm.Reds(row['Intensity'])),
                                               color=rgb2hex(cm.Reds(row['Intensity'])),
                                               popup=row['For Popup']).add_to(MAP), axis=1)
    
    return MAP
        
draw_map(data)